# 0.   Packages and Functions

In [1]:
!pip install pandas_datareader

In [83]:

#general
import pandas as pd
import numpy as np

#for dates and times
import random
import time
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

#prices
import pandas_datareader as web


In [84]:


def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [85]:
def weekend2weekday (transactions, col_date, col_day):


  dates2_BUY=list()
  days2_BUY=list()

  for index,row in transactions.iterrows():
    buy_date = datetime.datetime.strptime(row[col_date], "%Y-%m-%d")
    if row[col_day]=='Sat':
      buy_date2 = buy_date - datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Fri'
      days2_BUY.append(buy_day2)
    elif row[col_day]=='Sun':
      buy_date = datetime.datetime.strptime(row[col_day], "%Y-%m-%d")
      buy_date2 = buy_date + datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Mon'
      days2_BUY.append(buy_day2)
    else:
      buy_date2=row[col_date]
      dates2_BUY.append(buy_date2)
      buy_day2=row[col_day]
      days2_BUY.append(buy_day2)

    transactions['date_BUY_fix']=dates2_BUY
    transactions['day_BUY_fix']=days2_BUY


  return transactions

# 1. Setting investment parameters

## Temporal horizons
 In number of days
 - 1 day to 2 years

In [86]:
horizons=[1,2,3,4,5,6,7,10,15,20,30,40,50,60,70,80,90,100,120,150,180,210,240,270,300,330,360,420,480,540,600,660,720]


## Amounts in dollars
 - 50 to 50,000 dollars

In [87]:
amounts=[50,100,200,300,400,500,1000,1500,2000,3000,4000,5000,10000,15000,20000,25000,50000]

# 2. Data extraction

## Companies' stock prices

In [88]:
#Selecting a list of companies we are interested in analyzing:
companies = ['AAPL',
            'AMZN',
            'MSFT',
            'GOOG',
            'FB',
            'VISA',
            'NVDA',
            'PG',
            'TSLA']

We are going to extract info from 10 years from the companies selected

In [89]:
#Collecting the data of each company in a list
l_companies_prices=list()
error_companies=list()

for company in companies:
    try:
        prices= web.DataReader(company, data_source='yahoo', start='2010-10-10', end='2020-10-10')
        l_companies_prices.append(prices)
        print(company)
    
    except :
        print('There were  errors when extracting data of  ', company)
        #elimino de la lista esa empresa que no se consiguieron datos.
        error_companies.append(company)

for c in error_companies:
  companies.remove(c)
    

AAPL
AMZN
MSFT
GOOG
FB
There were  errors when extracting data of   VISA
NVDA
PG
TSLA


In [90]:
companies

['AAPL', 'AMZN', 'MSFT', 'GOOG', 'FB', 'NVDA', 'PG', 'TSLA']

In [91]:
len(l_companies_prices)

8

In [11]:
len(companies)

8

In [92]:
print(companies[1])
l_companies_prices[1].columns

AMZN


Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [13]:
company_pos=companies.index("AAPL")
l_companies_prices[company_pos].loc['2010-10-11','Close']

10.54857063293457

In [95]:
l_companies_prices[4].to_csv('PG_prices.csv')

In [96]:
l_companies_prices[4]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-05-18,45.000000,38.000000,42.049999,38.230000,573576400,38.230000
2012-05-21,36.660000,33.000000,36.529999,34.029999,168192700,34.029999
2012-05-22,33.590000,30.940001,32.610001,31.000000,101786600,31.000000
2012-05-23,32.500000,31.360001,31.370001,32.000000,73600000,32.000000
2012-05-24,33.209999,31.770000,32.950001,33.029999,50237200,33.029999
...,...,...,...,...,...,...
2020-10-05,264.869995,260.839996,262.200012,264.649994,12822300,264.649994
2020-10-06,265.690002,258.239990,261.779999,258.660004,18696900,258.660004
2020-10-07,260.179993,254.820007,259.209991,258.119995,23133400,258.119995


VOLATILITY

In [100]:
l_volatilities = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    log_ret = np.log(df_company_prices['Close'] / df_company_prices['Close'].shift(1))
    volatility = log_ret.rolling(window=252).std() * np.sqrt(252)
    df_company_prices.insert(2, "Volatility", volatility, True)
    l_volatilities.append(df_company_prices)




In [17]:
print(l_volatilities)

[                  High         Low  Volatility        Open       Close  \
Date                                                                     
2010-10-11   10.615714   10.521429         NaN   10.526429   10.548571   
2010-10-12   10.696429   10.446071         NaN   10.550357   10.662143   
2010-10-13   10.784286   10.707143         NaN   10.721429   10.719286   
2010-10-14   10.802500   10.728571         NaN   10.774643   10.796786   
2010-10-15   11.250000   10.889643         NaN   10.980000   11.240714   
...                ...         ...         ...         ...         ...   
2020-10-05  116.650002  113.550003    0.444997  113.910004  116.500000   
2020-10-06  116.120003  112.250000    0.446135  115.699997  113.160004   
2020-10-07  115.550003  114.129997    0.446117  114.620003  115.080002   
2020-10-08  116.400002  114.589996    0.446045  116.250000  114.970001   
2020-10-09  117.000000  114.919998    0.446154  115.279999  116.970001   

                 Volume   Adj Close 

In [101]:
l_returns = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    returns = (df_company_prices['Close'] - df_company_prices['Close'].shift(1)) / df_company_prices['Close'].shift(1)
    df_company_prices.insert(2, "returns", returns, True)
    l_returns.append(df_company_prices)

In [57]:
l_returns

[                  High         Low   returns   returns   returns  Volatility  \
 Date                                                                           
 2010-10-11   10.615714   10.521429       NaN       NaN       NaN         NaN   
 2010-10-12   10.696429   10.446071  0.010767  0.010767  1.010767         NaN   
 2010-10-13   10.784286   10.707143  0.005359  0.005359  1.005359         NaN   
 2010-10-14   10.802500   10.728571  0.007230  0.007230  1.007230         NaN   
 2010-10-15   11.250000   10.889643  0.041117  0.041117  1.041117         NaN   
 ...                ...         ...       ...       ...       ...         ...   
 2020-10-05  116.650002  113.550003  0.030791  0.030791  1.030791    0.444997   
 2020-10-06  116.120003  112.250000 -0.028669 -0.028669  0.971331    0.446135   
 2020-10-07  115.550003  114.129997  0.016967  0.016967  1.016967    0.446117   
 2020-10-08  116.400002  114.589996 -0.000956 -0.000956  0.999044    0.446045   
 2020-10-09  117.000000  114

yearly returns

In [102]:
l_yearly_returns = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    try:
        df_yearly = df_company_prices["Close"].resample('Y').first().pct_change()
        
        yearly_mean = df_yearly.mean()
    except:
        print( company)
        print(df_company_prices)
        yearly_mean = 0
    l_yearly_returns.append([company,yearly_mean])
print(l_yearly_returns)

[['AAPL', 0.24576212706845474], ['AMZN', 0.32878772362638664], ['MSFT', 0.21823741952413256], ['GOOG', 0.191378431933231], ['FB', 0.30161872761801845], ['NVDA', 0.4931171561846849], ['PG', 0.07725201668740694], ['TSLA', 0.5157369284319866]]


# 3. Random transactions

In [103]:
transactions=pd.DataFrame()
#we do 40,000 initial transactions

#id
transactions['id']=range(1,40001)

## Companies

In [104]:
#random company
transactions["company"]=np.random.choice(companies, size=len(transactions))

## Horizons

In [105]:
#random horizon
transactions["horizon"] = np.random.choice(horizons, size=len(transactions))


## Investment Amounts

In [106]:
#random investment amount
transactions["amount"] = np.random.choice(amounts, size=len(transactions))


## Dates - BUY

In [107]:
#random buy date
start='2013-10-10'
end='2018-10-10'

dates_buy=list()

for i in range(0, transactions.shape[0]):
    r_date=random_date(start,end,random.random())
    dates_buy.append(r_date)


print(len(dates_buy))
transactions['date_BUY']=dates_buy

40000


### Fixing dates (BUY)

In [108]:
day_names=list()

for d in dates_buy:

  dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=dt.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_BUY']=day_names


In [109]:
dates2_BUY=list()
days2_BUY=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
  if row['day_BUY']=='Sat':
    buy_date2 = buy_date - datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Fri'
    days2_BUY.append(buy_day2)
  elif row['day_BUY']=='Sun':
    buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
    buy_date2 = buy_date + datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Mon'
    days2_BUY.append(buy_day2)
  else:
    buy_date2=buy_date
    dates2_BUY.append(buy_date2)
    buy_day2=row['day_BUY']
    days2_BUY.append(buy_day2)


transactions['date_BUY_fix']=dates2_BUY
transactions['day_BUY_fix']=days2_BUY

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [110]:
#checking for holidays (NYSE is closed)

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_BUY:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_BUY']=l_holidays

In [111]:
transactions_h=transactions[transactions.holiday_BUY=='Holiday']

transactions=transactions[transactions.holiday_BUY!='Holiday']

## Prices - BUY

In [112]:
#price when bought

l_buy_prices=list()
l_volatility=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[buy_date,'Adj Close']
    volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
  except :
    price=0
    

  l_buy_prices.append(price)

transactions['price_BUY']=l_buy_prices


VOLATILITY BUY


In [113]:
l_volatility_buy=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  try:
    volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
  except :
    volatility=0
  l_volatility_buy.append(volatility)
print(len(l_volatility_buy))


transactions['Volatility_Buy'] = l_volatility_buy

38046


## Dates - SELL

In [118]:
#random sell date
dates_SELL=list()
expected_return = list()

for index,row in transactions.iterrows():
  comp=row['company']
  company_pos=companies.index(comp)
  buy_date = row['date_BUY_fix']
  sell_date = buy_date + datetime.timedelta(row['horizon'])
  expected_return.append(row['horizon']/365 * l_yearly_returns[company_pos][1])
  
  dates_SELL.append(sell_date)

transactions['expected_return (yearly)'] = expected_return
transactions['date_SELL']=dates_SELL

### Fixing dates (SELL)

In [115]:
day_names=list()



for d in dates_SELL:

  # dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=d.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_SELL']=day_names


In [124]:
dates2_SELL=list()
days2_SELL=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  SELL_date = row['date_SELL']
  if row['day_SELL']=='Sat':
    SELL_date2 = SELL_date - datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Fri'
    days2_SELL.append(SELL_day2)
  elif row['day_SELL']=='Sun':
    SELL_date = row['date_SELL']
    SELL_date2 = SELL_date + datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Mon'
    days2_SELL.append(SELL_day2)
  else:
    SELL_date2=SELL_date
    dates2_SELL.append(SELL_date2)
    SELL_day2=row['day_SELL']
    days2_SELL.append(SELL_day2)


transactions['date_SELL_fix']=dates2_SELL
transactions['day_SELL_fix']=days2_SELL

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

Sharpe Ratio


In [116]:
l_sharpe=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  r = l_returns
  try:
    df_company_returns = pd.DataFrame(l_returns[company_pos])
    r=df_company_returns["returns"]
    sr = r.mean()/r.std() * np.sqrt(252)
    sr= l_volatilities[company_pos].loc[buy_date, 'Volatility']
  except :
    sr=0
  l_sharpe.append(sr)
print(len(l_sharpe))


transactions['Sharpe Ratio'] = l_sharpe

38046


In [122]:
transactions

,id,company,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,Volatility_Buy,expected_return (yearly),date_SELL,day_SELL,Sharpe Ratio
0,1,AMZN,10,25000,2018-03-06,Tue,2018-03-06,Tue,,1537.640015,0.227187,0.009008,2018-03-16,Fri,0.227187
1,2,FB,15,400,2017-11-03,Fri,2017-11-03,Fri,,178.919998,0.178209,0.012395,2017-11-18,Sat,0.178209
2,3,AMZN,1,300,2017-04-29,Sat,2017-04-28,Fri,,924.989990,0.210226,0.000901,2017-04-29,Sat,0.210226
3,4,GOOG,360,3000,2014-07-22,Tue,2014-07-22,Tue,,593.111633,0.235331,0.188757,2015-07-17,Fri,0.235331
4,5,PG,15,50000,2014-11-12,Wed,2014-11-12,Wed,,74.582565,0.120184,0.003175,2014-11-27,Thu,0.120184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,GOOG,360,2000,2015-01-14,Wed,2015-01-14,Wed,,499.498627,0.212721,0.188757,2016-01-09,Sat,0.212721
39996,39997,NVDA,30,50000,2016-10-11,Tue,2016-10-11,Tue,,65.229042,0.347704,0.040530,2016-11-10,Thu,0.347704
39997,39998,TSLA,330,50000,2018-05-10,Thu,2018-05-10,Thu,,61.004002,0.399889,0.466283,2019-04-05,Fri,0.399889
39998,39999,PG,40,50,2014-08-10,Sun,2014-08-11,Mon,,67.397537,0.127641,0.008466,2014-09-20,Sat,0.127641


In [125]:
#checking for holidays (NYSE is closed)
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_SELL:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_SELL']=l_holidays

## Prices - SELL

In [126]:
#price when sold

l_sell_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[sell_date,'Adj Close']
  except :
    price=0

  l_sell_prices.append(price)

transactions['price_SELL']=l_sell_prices


VOLATILITY SELL

In [127]:
l_volatility_sell=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    volatility=l_volatilities[company_pos].loc[sell_date,'Volatility']
  except :
    volatility=0

  l_volatility_sell.append(volatility)

transactions['Volatility_sell']=l_volatility_sell

In [128]:
transactions_h=transactions[transactions.holiday_SELL=='Holiday']

transactions=transactions[transactions.holiday_SELL!='Holiday']

In [129]:
transactions

,id,company,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,Volatility_Buy,expected_return (yearly),date_SELL,day_SELL,Sharpe Ratio,date_SELL_fix,day_SELL_fix,holiday_SELL,price_SELL,Volatility_sell
0,1,AMZN,10,25000,2018-03-06,Tue,2018-03-06,Tue,,1537.640015,0.227187,0.009008,2018-03-16,Fri,0.227187,2018-03-16,Fri,,1571.680054,0.228277
1,2,FB,15,400,2017-11-03,Fri,2017-11-03,Fri,,178.919998,0.178209,0.012395,2017-11-18,Sat,0.178209,2017-11-17,Fri,,179.000000,0.170551
2,3,AMZN,1,300,2017-04-29,Sat,2017-04-28,Fri,,924.989990,0.210226,0.000901,2017-04-29,Sat,0.210226,2017-04-28,Fri,,924.989990,0.210226
3,4,GOOG,360,3000,2014-07-22,Tue,2014-07-22,Tue,,593.111633,0.235331,0.188757,2015-07-17,Fri,0.235331,2015-07-17,Fri,,672.929993,0.248367
5,6,PG,80,2000,2016-02-19,Fri,2016-02-19,Fri,,71.017471,0.166489,0.016932,2016-05-09,Mon,0.166489,2016-05-09,Mon,,71.888115,0.164962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,GOOG,360,2000,2015-01-14,Wed,2015-01-14,Wed,,499.498627,0.212721,0.188757,2016-01-09,Sat,0.212721,2016-01-08,Fri,,714.469971,0.288027
39996,39997,NVDA,30,50000,2016-10-11,Tue,2016-10-11,Tue,,65.229042,0.347704,0.040530,2016-11-10,Thu,0.347704,2016-11-10,Thu,,66.846718,0.328577
39997,39998,TSLA,330,50000,2018-05-10,Thu,2018-05-10,Thu,,61.004002,0.399889,0.466283,2019-04-05,Fri,0.399889,2019-04-05,Fri,,54.992001,0.586330
39998,39999,PG,40,50,2014-08-10,Sun,2014-08-11,Mon,,67.397537,0.127641,0.008466,2014-09-20,Sat,0.127641,2014-09-19,Fri,,69.870781,0.123412


In [130]:
transactions_ready=transactions[['company','horizon','amount','date_BUY_fix','date_SELL_fix','price_BUY','price_SELL','Volatility_Buy','Volatility_sell', 'Sharpe Ratio', 'expected_return (yearly)']]

In [131]:
transactions_ready

,company,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly)
0,AMZN,10,25000,2018-03-06,2018-03-16,1537.640015,1571.680054,0.227187,0.228277,0.227187,0.009008
1,FB,15,400,2017-11-03,2017-11-17,178.919998,179.000000,0.178209,0.170551,0.178209,0.012395
2,AMZN,1,300,2017-04-28,2017-04-28,924.989990,924.989990,0.210226,0.210226,0.210226,0.000901
3,GOOG,360,3000,2014-07-22,2015-07-17,593.111633,672.929993,0.235331,0.248367,0.235331,0.188757
5,PG,80,2000,2016-02-19,2016-05-09,71.017471,71.888115,0.166489,0.164962,0.166489,0.016932
...,...,...,...,...,...,...,...,...,...,...,...
39995,GOOG,360,2000,2015-01-14,2016-01-08,499.498627,714.469971,0.212721,0.288027,0.212721,0.188757
39996,NVDA,30,50000,2016-10-11,2016-11-10,65.229042,66.846718,0.347704,0.328577,0.347704,0.040530
39997,TSLA,330,50000,2018-05-10,2019-04-05,61.004002,54.992001,0.399889,0.586330,0.399889,0.466283
39998,PG,40,50,2014-08-11,2014-09-19,67.397537,69.870781,0.127641,0.123412,0.127641,0.008466


## Filtering transactions with error (Price = 0)

In [132]:
price_error=transactions_ready[(transactions_ready.price_BUY==0) | (transactions_ready.price_SELL==0)]
print('with errors: ', price_error.shape)
print(transactions_ready.shape)

transactions_ready=pd.concat([transactions_ready, price_error, price_error]).drop_duplicates(keep=False)
print(transactions_ready.shape)

with errors:  (399, 11)
(36281, 11)
(35642, 11)


# Real Profit/Loss (Rentabilidad Real)

In [133]:
transactions_ready['nominal_return']=(transactions_ready['price_SELL']-transactions_ready['price_BUY'])/transactions_ready['price_BUY']

In [134]:
transactions_ready

,company,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),nominal_return
0,AMZN,10,25000,2018-03-06,2018-03-16,1537.640015,1571.680054,0.227187,0.228277,0.227187,0.009008,0.022138
1,FB,15,400,2017-11-03,2017-11-17,178.919998,179.000000,0.178209,0.170551,0.178209,0.012395,0.000447
2,AMZN,1,300,2017-04-28,2017-04-28,924.989990,924.989990,0.210226,0.210226,0.210226,0.000901,0.000000
3,GOOG,360,3000,2014-07-22,2015-07-17,593.111633,672.929993,0.235331,0.248367,0.235331,0.188757,0.134576
5,PG,80,2000,2016-02-19,2016-05-09,71.017471,71.888115,0.166489,0.164962,0.166489,0.016932,0.012260
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,GOOG,360,2000,2015-01-14,2016-01-08,499.498627,714.469971,0.212721,0.288027,0.212721,0.188757,0.430374
39996,NVDA,30,50000,2016-10-11,2016-11-10,65.229042,66.846718,0.347704,0.328577,0.347704,0.040530,0.024800
39997,TSLA,330,50000,2018-05-10,2019-04-05,61.004002,54.992001,0.399889,0.586330,0.399889,0.466283,-0.098551
39998,PG,40,50,2014-08-11,2014-09-19,67.397537,69.870781,0.127641,0.123412,0.127641,0.008466,0.036696


In [135]:
transactions_ready['investment']=''
investments=list()
for index, row in transactions_ready.iterrows():
  if row['nominal_return']>0:
    inv='GOOD'
    investments.append(inv)
  elif row['nominal_return']<0:
    inv='BAD'
    investments.append(inv)
  else:
    inv='0'
    investments.append(inv)

transactions_ready['investment']=investments

In [44]:
transactions_ready.shape

(35481, 9)

In [136]:
transactions_ready

,company,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),nominal_return,investment
0,AMZN,10,25000,2018-03-06,2018-03-16,1537.640015,1571.680054,0.227187,0.228277,0.227187,0.009008,0.022138,GOOD
1,FB,15,400,2017-11-03,2017-11-17,178.919998,179.000000,0.178209,0.170551,0.178209,0.012395,0.000447,GOOD
2,AMZN,1,300,2017-04-28,2017-04-28,924.989990,924.989990,0.210226,0.210226,0.210226,0.000901,0.000000,0
3,GOOG,360,3000,2014-07-22,2015-07-17,593.111633,672.929993,0.235331,0.248367,0.235331,0.188757,0.134576,GOOD
5,PG,80,2000,2016-02-19,2016-05-09,71.017471,71.888115,0.166489,0.164962,0.166489,0.016932,0.012260,GOOD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,GOOG,360,2000,2015-01-14,2016-01-08,499.498627,714.469971,0.212721,0.288027,0.212721,0.188757,0.430374,GOOD
39996,NVDA,30,50000,2016-10-11,2016-11-10,65.229042,66.846718,0.347704,0.328577,0.347704,0.040530,0.024800,GOOD
39997,TSLA,330,50000,2018-05-10,2019-04-05,61.004002,54.992001,0.399889,0.586330,0.399889,0.466283,-0.098551,BAD
39998,PG,40,50,2014-08-11,2014-09-19,67.397537,69.870781,0.127641,0.123412,0.127641,0.008466,0.036696,GOOD


In [46]:
transactions_ready.investment.value_counts()

GOOD    24662
BAD     10478
0         341
Name: investment, dtype: int64

## Variables calculation